### Attributes defination:
1. Subject (IoT device or user) attributes
- role: employee, visitor, security staff, admin
- department: HR, Engineering, Maintenance
- securityLevel: 1–5
- deviceType: sensor, actuator, gateway
- location: floor number or zone

2. Object (resource/device) attributes
- resourceType: temperatureSensor, doorLock, camera, HVAC
- sensitivityLevel: low, medium, high
- location: same zone/floor

3. Environmental attributes
- time: working hours or not
- alertState: normal, emergency, lockdown
- motionDetected: yes/no

### Policy defination


---
During the registration phase, an IoT device initiates the onboarding process by submitting a registration request containing basic device information, such as its identifier and manufacturer-related metadata, to a nearby fog node. To protect sensitive registration data from exposure, the fog node forwards the received information to its Trusted Execution Environment (TEE) enclave using a secure channel encrypted with the enclave’s public key.

Inside the TEE, the fog node verifies the device’s legitimacy by validating manufacturer certificates, firmware signatures, and administrative records with the assistance of attribute authorities. Based on the verified evidence, the enclave assigns a set of attributes to the IoT device, including static attributes (e.g., device type and manufacturer), administrative attributes (e.g., ownership domain and deployment zone), and dynamic attributes (e.g., security level). To ensure integrity and auditability, cryptographic hashes of the assigned attributes, together with issuer information and timestamps, are recorded on the blockchain through the attribute provider contract.

The complete attribute set is then encrypted using an enclave-protected symmetric key and stored locally on the fog node outside the enclave, preventing leakage even if the fog operating system is compromised. To enhance system reliability and fault tolerance, the fog node periodically backs up the encrypted attributes to a trusted cloud-based data storage service. Finally, upon successful completion of the registration process, the fog node notifies the IoT device of its registration status and enables it to request access to system resources in subsequent phases.


Registration phase:

We use elliptic curve encryption to ensure security communication. 

Step 1 — Registration Request by IoT Device  
  IoT sends basic identity info:  
 - deviceID
 - manufacturerID
 - model
 - firmwareVersion
 - MAC / serial  
These are claims, not trusted yet.

Step 2 — Secure Channel Establishment  
IoT encrypts registration data using fog TEE public key:
 - C = Enc(PK_fogTEE, registration_data)  
Only TEE can decrypt → protects against compromised fog OS.

Step 3 — Evidence Verification Inside Fog TEE  
Fog TEE verifies: 
 - Manufacturer signature on firmware
 - Certificate chain validity
 - Device uniqueness
 - Network consistency  
If verification fails → registration rejected.

Step 4 — Attribute Authority Decision Logic  
The AA (logically executed in fog TEE or coordinated AAs) assigns attributes based on verified evidence:  
Example decision table:
| Evidence                    | Assigned Attribute        |
| --------------------------- | ------------------------- |
| Verified firmware signature | `deviceType = sensor`     |
| Admin deployment record     | `ownerDomain = FactoryA`  |
| Fog cluster location        | `location = FogCluster03` |
| IDS benign behavior         | `securityLevel = 0.75`    |
  
Attributes are never inferred from device claims alone.

Step 5 - Attribute Blinding  
AA bind the set of attributes with deviceID then sign it to proove certification.  

Step 6 Storage  
Attributes and access policies are encrypted using a symmetric key protected by the fog node’s TEE and stored outside the enclave on the fog node. To prevent data loss caused by fog node failures, the fog node periodically backs up authorization state to a trusted cloud-based DDS.

During backup, the TEE decrypts the local data, attaches metadata including owner identity, version number, and timestamp, and transmits the data to the cloud together with a TEE-generated signature. Upon receipt, the cloud verifies the signature and freshness before storing the backup data. This design enables efficient recovery while preserving integrity and authenticity of authorization information.

To ensure availability and fault tolerance, the fog node periodically backs up encrypted attributes and access policies to a trusted cloud-based DDS. The backup process is executed within the fog node’s TEE, which decrypts locally stored data using a sealed symmetric key, attaches metadata including owner identity, version number, and timestamp, and encrypts the backup payload for the cloud. The TEE further signs the encrypted payload to guarantee authenticity and integrity. Upon receiving the backup, the cloud verifies the signature and freshness before storing the data, enabling secure and efficient recovery in the event of fog node failure.

---
Upon receiving an access request, the fog node first authenticates the requesting IoT device by verifying its digital signature and checks request freshness using timestamps or nonces to prevent replay attacks. The fog node then verifies the existence and validity of the requested resource and its associated access policy before proceeding. Once these preliminary checks are completed, the fog node forwards the encrypted subject attributes and resource access policy to the Trusted Execution Environment (TEE) for authorization.

Within the enclave, the TEE securely decrypts the protected attributes and policies using enclave-bound keys and evaluates the access request according to predefined Attribute-Based Access Control (ABAC) rules. Because all sensitive authorization logic and data processing are confined within the enclave, neither a compromised fog operating system nor malicious insiders can manipulate attributes, alter policies, or influence authorization outcomes. The resulting access decision is then returned to the fog node for enforcement, enabling secure, fine-grained, and tamper-resistant authorization at the network edge

## Architecture  
1. Fog node: They are partially trusted, means they may be OS-level compromised or exist malicious insiders, but the Trusted Execution Environment on each fog node is assumed to be secure and provides hardware-enforced isolation. Fog nodes in each domain jointly maintain a Blockchain and manage Iot devices in the domain. It deploys TEE and runs smart contracts in the enclave. It can be treated as a Attribute Authority in the ABAC system. It also store registered iots' info(attributes/access policies) securitly and periodically back up those info to cloud.
2. Iot node: They are iot devices in the domain and untrusted. The computing and storage capacity is relatively weaker than the Fogs. It may ask access of nodes or resources inide or outside its own domain.
3. Cloud/DDS: The cloud-based Dedicated Data Storage (DDS) acts as a trusted, highly available backend storage service. Unlike traditional centralized authorization servers, the cloud/DDS does not participate in real-time authentication or authorization decisions. Instead, it provides durability, recoverability, and global availability for authorization-related data. It also play the role of Resource Provider (application services, analytics, historical data, resources that IoT devices may want to access).
4. blockchain: Serve as a global distributed ledger, the blockchain in this system carries the information of different domains which will be used for cross-domain authentication. The registered iots' identity information will be stored on the blockchain for later authentication. IoT's revoking information will also be placed on the.